In [ ]:
# Actualizamos paquetes críticos y listamos todos los instalados en la VM

In [ ]:
%%bash 
pip install sklearn --upgrade
pip install --upgrade tensorflow
pip freeze

In [ ]:
"""
Ejemplo 1: llamada a la API de BQ 
Extraído de los notebooks de muestra de Datalab 
"""

In [ ]:
import google.datalab.bigquery as bq

# Hacemos una query para obtenerlos nacimientos de varones cada año
total_births = bq.Query('SELECT CAST(source_year AS string) AS year, COUNT(is_male) AS birth_count FROM `publicdata.samples.natality` GROUP BY year ORDER BY year DESC LIMIT 15')
df = total_births.execute(output_options=bq.QueryOutput.dataframe()).result()

# Vemos las 10 primeras filas del dataframe
df.head(10)


In [ ]:

# Ploteamos la query
ax = df.plot(kind='bar',x='year',title='Total births by year')
ax.set_xlabel('Year')
ax.set_ylabel('Birth count')

# Hacemos una query para obtenerlos nacimientos de niños cada día de la semana
births_by_weekday = bq.Query('SELECT CAST(wday AS string) AS weekday, SUM(CASE WHEN is_male THEN 1 ELSE 0 END) AS male_births, SUM(CASE WHEN is_male THEN 0 ELSE 1 END) AS female_births FROM `publicdata.samples.natality` WHERE wday IS NOT NULL GROUP BY weekday ORDER BY weekday ASC')
df2 = births_by_weekday.execute(output_options=bq.QueryOutput.dataframe()).result()

# Ploteamos la query
df2 = births_by_weekday.execute(output_options=bq.QueryOutput.dataframe()).result()
ax = df2.plot(kind='line',x='weekday',title='Births by weekday')
ax.set_xlabel('Weekday')
ax.set_ylabel('Total')

In [ ]:
"""
Ejemplo 2: llamada llamada 'mágica' a BQ
Usamos la tabla de BQ de ejemplo con los viajes en taxi en Chicago desde 2013 (105 millones de filas - 36GB)
Intentaremos predecir el precio del viaje en taxi en función de las coordenadas donde se recoge y se deja al cliente, la duración y la distancia del viaje
"""

In [ ]:
from __future__ import division

import google.datalab.bigquery as bq
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
%%bq query -n taxi_queries
SELECT trip_seconds, trip_miles, pickup_latitude, pickup_longitude, dropoff_latitude, dropoff_longitude, fare
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE trip_miles IS NOT NULL AND
      trip_seconds IS NOT NULL AND
      pickup_latitude IS NOT NULL AND 
      pickup_longitude IS NOT NULL AND
      dropoff_latitude IS NOT NULL AND
      dropoff_longitude IS NOT NULL AND  
      fare IS NOT NULL
LIMIT 300000



In [ ]:
# Ejecutamos la query
df = taxi_queries.execute(output_options=bq.QueryOutput.dataframe()).result()

In [ ]:
df.head()

In [ ]:
# Normalización de columnas (normalizamos todo el dataset, por lo que no tenemos que guardar la media ni la desviación)

cols_to_norm = ['pickup_latitude','pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']
df[cols_to_norm] = df[cols_to_norm].apply(lambda x: (x - x.mean()) / x.std())

df.head()

In [ ]:
ax = df.plot(kind='scatter',x='pickup_longitude', y='pickup_latitude', title='Pickup locations')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

In [ ]:
ax = df.plot(kind='scatter',x='dropoff_longitude', y='dropoff_latitude', title='Dropoff locations')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

In [ ]:
ax = df['fare'].hist()
ax.set_xlabel('Fares')
ax.set_ylabel('Number of trips')
ax.set_title('Fares histogram')

In [ ]:
# Veamos tras aplicar algoritmos de reducción de dimensionalidad si los viajes con tarifas más comunes son "diferentes" del resto (es decir, si no tenemos que hacer feature engineering)

from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE

df_aux = df.sample(500)

# Consideramos tarifas "normales" entre $4.8 y $7.5
df_aux['normal_fare'] = df_aux['fare'].apply(lambda x: 1 if x >= 4.8 and x <= 7.5  else 0)
df_aux.drop('fare', axis=1, inplace=True)

# Nos quedamos con la columna de etiquetas
y = df_aux.ix[:,-1].values

df_aux.head()

In [ ]:
df_aux['normal_fare'].describe()

In [ ]:
#Construimos un gráfico con los puntos obtenidos al aplicar tsne o PCA
def plot_data_2d(X, y, x_lims = None, y_lims = None):
  color_map = {0:'red', 1:'blue'}
  plt.figure()
  for idx, cl in enumerate(np.unique(y)):
    plt.scatter(x = X[y==cl,0], 
                y = X[y==cl,1], 
                c = color_map[idx], 
                label = cl)
  plt.xlabel('X in t-SNE')
  plt.ylabel('Y in t-SNE')
  plt.legend(loc='upper left')
  plt.title('t-SNE visualization of Chicago taxi trips')
  if x_lims is not None and y_lims is not None:
    plt.xlim(x_lims)
    plt.ylim(y_lims)
  plt.show()
  

In [ ]:
# Usamos PCA en este caso
from sklearn import decomposition

pca = decomposition.PCA(n_components=2)
pca.fit(df_aux)
X = pca.transform(df_aux)

x_lims = (X[:,0].mean()-1000,X[:,0].mean()+1000)
y_lims = (X[:,1].mean()-10,X[:,1].mean()+10)

plot_data_2d(X, y, x_lims, y_lims)

In [ ]:
# Creamos una visualización en 2d con el algoritmo t-SNE para ver la diferencia entre viajes con tarifas "más comunes" y el resto
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE


# Scale features to improve the training ability of TSNE.
standard_scaler = StandardScaler()
df_std = standard_scaler.fit_transform(df_aux)

# Compute t-SNE algorithm
tsne = TSNE(n_components=2, random_state=0)
tsne_2d = tsne.fit_transform(df_std)

#Construimos un gráfico con los puntos obtenidos al aplicar tsne
plot_data_2d(tsne_2d, y)

In [ ]:
import numpy as np
import sklearn.model_selection as ms

TRAIN_SIZE = 0.8
TEST_SIZE = 0.1
VAL_SIZE = 0.1

dataset = df.as_matrix()

X_train, X_test, Y_train, Y_test = ms.train_test_split(dataset[:,:-1], dataset[:,-1], test_size=TEST_SIZE, random_state=1)

X_train, X_val, Y_train, Y_val = ms.train_test_split(X_train, Y_train, test_size=VAL_SIZE/TRAIN_SIZE, random_state=1)

In [ ]:
import numpy as np
#import tensorflow
#import tensorflow as tf
import sys
import tensorflow.contrib.keras as keras

In [ ]:
#from tensorflow.contrib.keras import *
from tensorflow.contrib.keras.python.keras.regularizers import l1,l2
from tensorflow.contrib.keras.python.keras.models import Sequential, load_model
from tensorflow.contrib.keras.python.keras.layers import Dense, Dropout, Activation
from tensorflow.contrib.keras.python.keras.constraints import max_norm
from tensorflow.contrib.keras.python.keras.optimizers import RMSprop, Adam
from tensorflow.contrib.keras.python.keras.layers.normalization import BatchNormalization
import os

# Disable info warnings from TF
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'




In [ ]:

# Volvemos a limitar el resultado de la query
LIMIT = 10000

# Limitamos los ejemplos de entrenamiento a viajes de menos de 20$

mask_train = Y_train < 20
mask_val = Y_val < 20
mask_test = Y_test < 20


X_train = X_train[mask_train]
Y_train = Y_train[mask_train]
X_val = X_val[mask_val]
Y_val = Y_val[mask_val]
X_test = X_test[mask_test]
Y_test = Y_test[mask_test]


# Hyperparameters
batch_size = 500
epochs = 100
dropout_rate = 0.5

TRAIN_LIMIT = int(LIMIT*TRAIN_SIZE)
VAL_LIMIT = int(LIMIT*VAL_SIZE)
TEST_LIMIT = int(LIMIT*TEST_SIZE)

x_train = X_train[:TRAIN_LIMIT]
y_train = Y_train[:TRAIN_LIMIT]
x_val = X_val[:VAL_LIMIT]
y_val = Y_val[:VAL_LIMIT]
x_test = X_test[:TEST_LIMIT]
y_test = Y_test[:TEST_LIMIT]

# Tenemos 6 neuronas de entrada
input_dim = dataset.shape[1] - 1

model = Sequential()
model.add(Dense(2,input_shape=(input_dim,), kernel_initializer='he_normal'))
#model.add(Dense(3,input_shape=(input_dim,)))

model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(Dropout(dropout_rate))
model.add(Dense(1))

model.summary()

model.compile(loss='mean_squared_error',
              optimizer=Adam(),
              metrics=[keras.metrics.mean_absolute_error, keras.metrics.mean_absolute_percentage_error])

# Entrenamos el modelo (con verbose 0 apagamos el log y con 1 se muestra con barra de progreso)
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=0,
                    validation_data=(x_val, y_val))


score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss (mse):', score[0])
print('Test mae:', score[1])
print('Test mape:', score[2])

y_pred = model.predict_proba(x_test, verbose = 0)




In [ ]:
import matplotlib.pyplot as plt

PLOT_LIM = 15
# Plot para ver como se desvían nuestras prediciones
fig, ax = plt.subplots()
ax.scatter(y_test[:LIMIT], y_pred[:LIMIT] )
ax.plot([0, PLOT_LIM], [0, PLOT_LIM], 'k--', lw=3)
ax.set_xlim([0, PLOT_LIM])
ax.set_ylim([0, PLOT_LIM])
ax.set_title('Chicago taxi trips prediction')
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

In [ ]:
y_pred = model.predict_proba(x_test[49:50], verbose = 0)

In [ ]:
y_pred

In [ ]:
y_test[49:50]

In [ ]:
abs(y_pred - y_test[50:51])